# Example on using RDKit, clustering techniques and xTB to find relevant conformers

Based on conformer selection by **ReSCoSS, Anikó Udvarhelyi, Stephane Rodde, Rainer Wilcken, J. CAMD 2021, 10.1007/s10822-020-00337-7**

An example flow for generating shape-diverse conformers.

- Generate many conformers with RDKit
- Use clustering technique and cheap shape description to filter conformers
- Use xTB to optimize conformers
- Use same clustering technique to find unique conformers

You now have a series of local-minima conformers based on xTB energy model


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import sys

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import SVG

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from rdkit.Chem import AllChem, PandasTools
from rdkit.Chem.Draw import MolsToGridImage, MolToImage, rdMolDraw2D

In [ ]:
from sklearn.cluster import DBSCAN, KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

In [ ]:
import collections
import functools
import logging
import sys

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import patheffects

In [ ]:
try:
    import ppqm
except ModuleNotFoundError:
    import pathlib

    cwd = pathlib.Path().resolve().parent
    sys.path.append(str(cwd))
    import ppqm

In [ ]:
from ppqm import chembridge
from ppqm import jupyter as ppqm_jupyter

In [ ]:
import rmsd

In [ ]:
# Set logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger("ppqm").setLevel(logging.INFO)
logging.getLogger("xtb").setLevel(logging.INFO)

In [ ]:
_logger = logging.getLogger(__name__)

In [ ]:
# Set DataFrames visuals
PandasTools.RenderImagesInAllDataFrames(images=True)

pd.options.display.max_rows = 60
pd.options.display.max_columns = 60
pd.options.display.float_format = '{:,.2f}'.format

# Plot functions

In [ ]:
def get_plot(n_ax=1):
    """ Get a jupyter-sized plot """
    fig, axs = plt.subplots(1, n_ax, sharey=True, sharex=True, figsize=(12, n_ax * 12))
    return fig, axs


def view_cluster(ax, values, cluster_indices, outliers=None, markersize=6):
    """ View clusters of points """

    n_clusters = len(cluster_indices)
    colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, n_clusters)]
    # colors = [plt.cm.rainbow(each) for each in np.linspace(0, 1, n_clusters)]

    for idxs, color in zip(cluster_indices, colors):

        marker_options = {
            "markerfacecolor": tuple(color),
            "markeredgecolor": "k",
            "markersize": markersize,
        }

        x = values[0, idxs]
        y = values[1, idxs]
        ax.plot(x, y, "o", **marker_options)

    if outliers is not None:

        outlier_color = [1, 1, 1, 1]
        marker_options = {
            "markerfacecolor": tuple(outlier_color),
            "markeredgecolor": "k",
            "markersize": markersize,
        }

        x = values[0, outliers]
        y = values[1, outliers]
        ax.plot(x, y, "o", **marker_options)


def hexbin(ax, xvalues, yvalues, density=25, mincount=2, colormap="PuRd", bins="log"):  # 'Greys'
    """ Wrapper for MPL hexbin func with sane defaults """

    # Settings
    lineswidth = 0.0  # white lines
    lineswidth = 0.2  # perfect fit
    lineswidth = 0.3  # fit for pngs
    lineswidth = 0.4  # fit for pngs

    hexbinpar = {
        "gridsize": density,
        "cmap": colormap,
        "linewidths": lineswidth,
        "mincnt": 1,
        "bins": bins,
    }

    _ = ax.hexbin(xvalues, yvalues, **hexbinpar)

    return


def fix_borders(ax, visibles=[False, False, True, True], fix_bounds=True):
    """ Make border pretty """

    directions = ["top", "right", "bottom", "left"]

    spines = ax.spines.items()
    spines = dict(spines)

    xticks = ax.get_xticks()
    yticks = ax.get_yticks()
    min_x, max_x = ax.get_xlim()
    min_y, max_y = ax.get_ylim()

    # Correct to the actual ticks
    (x_idxs,) = np.where((xticks > min_x) & (xticks < max_x))
    (y_idxs,) = np.where((yticks > min_y) & (yticks < max_y))
    xticks = xticks[x_idxs]
    yticks = yticks[y_idxs]

    min_x = np.min(xticks)
    max_x = np.max(xticks)

    min_y = np.min(yticks)
    max_y = np.max(yticks)

    for direction, visible in zip(directions, visibles):

        spine = spines[direction]
        spine.set_visible(visible)

        if not visible:
            continue

        if not fix_bounds:
            continue

        if direction == "left" or direction == "right":
            spine.set_bounds(min_y, max_y)

        else:
            spine.set_bounds(min_x, max_x)

# Settings

In [ ]:
N_CORES = 8

## Define a Molecule

In [ ]:
smiles = "Cc1ccc(-c2ncccn2)c(C(=O)N2CC(COc3ccc(F)cn3)CCC2C)c1"  # 10.1007/s10822-020-00337-7

In [ ]:
molobj = Chem.MolFromSmiles(smiles)

In [ ]:
molobj

# Generate a lot of conformers

Generate as many conformers as possible. Use your favorite program. We pretend that all conformers are covered by this. 

In this example we use RDKit ETKDGv3 to generate conformers


In [ ]:
%%time

n_conformers = 1_000
molobj_3d = Chem.Mol(molobj, True)
molobj_3d = Chem.AddHs(molobj_3d)
embed_parameters = rdDistGeom.ETKDGv3()
rdDistGeom.EmbedMultipleConfs(molobj_3d, n_conformers, embed_parameters)

pass

In [ ]:
molobj_3d.GetNumConformers()

In [ ]:
# ppqm_jupyter.show_molobj(molobj_3d)

# Look at the shape distribution 


There are *MANY* ways to look at the shape of a molecule. ResCoSS looks at a list of properties calculates by cosmotherm, including hydrogen-doner/acceptor.

For speed-reasons, we select two of the properties

- Solvent acessible Surface Area (SASA)
- Dipole moment

and calculate them by RDKit

    rdFreeSASA.CalcSASA(molobj, radii, confIdx=i)
    AllChem.ComputeGasteigerCharges(molobj)

which is rough estimates of QM properties, however good enough to describe if two conformers are shapely similar.


In [ ]:
def calculate_shape(mol):
    """ return (2, n_conformer) array with SASA and Dipole moments """
    
    # Get surface accessible solvent area
    sasas = ppqm.chembridge.get_sasa(mol, extra_radius=1.0)

    # Dipole moments
    dipole_moments = ppqm.chembridge.get_dipole_moments(mol)

    # Package in one numpy array
    values = [sasas, dipole_moments]
    values = np.asarray(values, dtype=np.float64)
    
    return values

In [ ]:
%%time
shapes_unfiltered = calculate_shape(molobj_3d)

In [ ]:
fig, axs = get_plot()
axs.plot(*shapes_unfiltered, "ko", ms=5)
axs.set_xlabel("SASA")
axs.set_ylabel("Dipole")

#for idx, (x, y) in enumerate(zip(*conformer_shapes)):
#    txt = plt.text(x, y, idx, fontsize=14)
#    txt.set_path_effects([patheffects.withStroke(linewidth=3, foreground="w")])

fix_borders(axs)

# Optimize all conformers

In [ ]:
def optimize_axyzc(
    atoms,
    coordinates,
    charge,
    calculation_options={},
    xtb_options={}
) -> np.array:
    """
    Optimize conformer with fast quantum chemistry

    - optimize with XTB2
    - if not_converged
      restart with XTB0 with n_steps
      optimize with XTB2
    - if not_converged
      optimize with cartisian coordinates
    - if not_converged
      return None

    :return coordiantes: Nx3 Array of coordiantes
    """

    assert isinstance(atoms[0], str), "func requires atoms in string format"

    options_default = {
        "opt": None,
        "cycles": 100,
    }
    options_default = dict(collections.ChainMap(options_default, calculation_options))

    options_fast = {
        "gfn": 0,
        "opt": None,
        "cycles": 200,
    }
    options_fast = dict(collections.ChainMap(options_fast, calculation_options))

    options_lax = {
        "opt": "lax",
    }
    options_fast = dict(collections.ChainMap(options_lax, calculation_options))

    # Start calculating with the defined options

    properties = ppqm.xtb.get_properties_from_axyzc(
        atoms, coordinates, charge, options=options_default, **xtb_options
    )

    if properties and properties[ppqm.xtb.COLUMN_CONVERGED]:
        return properties[ppqm.xtb.COLUMN_COORD], properties[ppqm.xtb.COLUMN_ENERGY]

    properties = ppqm.xtb.get_properties_from_axyzc(
        atoms, coordinates, charge, options=options_fast, **xtb_options
    )

    if not properties or ppqm.xtb.COLUMN_COORD not in properties:
        return None, None

    fast_coordinates = properties[ppqm.xtb.COLUMN_COORD]

    properties = ppqm.xtb.get_properties_from_axyzc(
        atoms, fast_coordinates, charge, options_default, **xtb_options
    )

    if properties and properties[ppqm.xtb.COLUMN_CONVERGED]:
        return properties[ppqm.xtb.COLUMN_COORD], properties[ppqm.xtb.COLUMN_ENERGY]

    if properties[ppqm.xtb.COLUMN_COORD] is None:
        return None, None

    fast_coordinates = properties[ppqm.xtb.COLUMN_COORD]

    properties = ppqm.xtb.get_properties_from_axyzc(
        atoms, fast_coordinates, charge, options=options_lax, **xtb_options
    )

    if not properties or properties[ppqm.xtb.COLUMN_COORD] is None:
        return None, None

    return properties[ppqm.xtb.COLUMN_COORD], properties[ppqm.xtb.COLUMN_ENERGY]

In [ ]:
def optimize_acxyz(atoms, charge, coordinates, **kwargs):
    """ Meta function for parallel func mapping """
    return optimize_axyzc(atoms, coordinates, charge, **kwargs)

In [ ]:
def optimize_molobj(
    molobj,
    show_progress=True,
    n_cores=1,
    scr=None,
    calculation_options={},
    xtb_options={},
    rmsd_threshold=3.0,
):
    """
    Optimize all the conformers in molobj.

    :param rmsd_threshold: Check if molcule moved away from local minima.
    """
    n_atoms = molobj.GetNumAtoms()
    molobj_prime = chembridge.copy_molobj(molobj)
    energies = []

    atoms, _, charge = chembridge.get_axyzc(molobj, atomfmt=str)

    n_conformers = molobj.GetNumConformers()
    coordinates_list = [
        np.asarray(conformer.GetPositions()) for conformer in molobj.GetConformers()
    ]

    n_procs = min(n_cores, n_conformers)
    results = []

    if "n_cores" in xtb_options:
        del xtb_options["n_cores"]

    if scr:
        xtb_options["scr"] = scr

    func = functools.partial(
        optimize_acxyz,
        atoms,
        charge,
        calculation_options=calculation_options,
        xtb_options=xtb_options,
    )

    results = ppqm.misc.func_parallel(
        func,
        coordinates_list,
        n_cores=n_procs,
        show_progress=show_progress,
        title="Optimize",
    )
    
    for idx, (coord, energy) in enumerate(results):

        # if conformer is unconverged, ignore
        if coord is None or energy is None:
            _logger.warning(f"conformer {idx} unconverged")
            continue

        # if conformer has changed a lot, warn
        original = chembridge.get_coordinates(molobj, confid=idx)
        displacement = rmsd.kabsch_rmsd(coord, original)
        if displacement > rmsd_threshold:
            _logger.warning(f"conformer {idx} has large displacement")

        # Molecule is converged, add to new molobj
        conformer = Chem.Conformer(n_atoms)
        chembridge.conformer_set_coordinates(conformer, coord)
        molobj_prime.AddConformer(conformer, assignId=True)
        energies.append(energy)

    energies = np.asarray(energies)

    return molobj_prime, energies

# Filter a lot of conformers to unique

In [ ]:
def find_conformer_groups(values):
    """ Unbound filter for clustered conformer shapes
    
    args:
        values - n_descriptors x n_conformers
    
    """
    
    dbscan_eps = 0.2
    dbscan_min_samples = 2
    
    # Transform to row-based descriptions
    values = values.T

    if values.shape[0] == 0:
        return []
    
    # Normalize the shape descriptors
    values = StandardScaler().fit_transform(values)
    
    # Find outliers
    db = DBSCAN(eps=dbscan_eps, min_samples=dbscan_min_samples).fit(values)
    labels = db.labels_
    cluster_labels, cluster_counts = np.unique(labels, return_counts=True)

    cluster_indices = []
    outliers = []
    for label in cluster_labels:
        (idxs,) = np.where(labels == label)

        if label == -1:
            outliers = idxs
        else:
            cluster_indices.append(idxs)
    
    return cluster_indices, outliers


def select_n(clusters, n=1):
    """ Select n indicies in clusters """
    
    indices = []
    
    for cluster in clusters:
        idxs = cluster[:n]
        indices += list(idxs)

    return indices

In [ ]:
# visualize the groups

In [ ]:
groups_indices, groups_outliers = find_conformer_groups(shapes_unfiltered)

In [ ]:
selected_indices = select_n(groups_indices) + list(groups_outliers)

In [ ]:
fig, ax = get_plot()

view_cluster(ax, shapes_unfiltered, groups_indices, outliers=groups_outliers)

ax.plot(
    shapes_unfiltered[0, selected_indices],
    shapes_unfiltered[1, selected_indices],
    "Xk",
    markersize=14,
    path_effects=[patheffects.withStroke(linewidth=4, foreground="w")],
)

fix_borders(ax)

In [ ]:
molobj_filtered = ppqm.chembridge.molobj_select_conformers(molobj_3d, selected_indices)

In [ ]:
molobj_filtered.GetNumConformers()

# Molecule with shape diverse conformers

In [ ]:
molobj_filtered_xtb, _ = optimize_molobj(molobj_filtered, n_cores=N_CORES)

In [ ]:
shapes_filtered_optimized = calculate_shape(molobj_filtered_xtb)

In [ ]:
fig, ax = get_plot()
ax.plot(*shapes_filtered_optimized, "ko", ms=5)
ax.set_xlabel("SASA")
ax.set_ylabel("Dipole")
fix_borders(ax)

In [ ]:
groups_indices, groups_outliers = find_conformer_groups(shapes_filtered_optimized)

In [ ]:
selected_indices = select_n(groups_indices) + list(groups_outliers)

In [ ]:
fig, ax = get_plot()

view_cluster(ax, shapes_filtered_optimized, groups_indices, outliers=groups_outliers)

ax.plot(
    shapes_unfiltered[0, selected_indices],
    shapes_unfiltered[1, selected_indices],
    "Xk",
    markersize=14,
    path_effects=[patheffects.withStroke(linewidth=4, foreground="w")],
)

fix_borders(ax)

In [ ]:
ppqm_jupyter.show_molobj(molobj_filtered_xtb)

# Backup slides

In [ ]:
%%time
molobj_3d_xtb, energies = optimize_molobj(molobj_3d, n_cores=N_CORES)

In [ ]:
%%time
shapes_unfiltered_optimized = calculate_shape(molobj_3d_xtb)

In [ ]:
fig, ax = get_plot()
ax.plot(*shapes_unfiltered_optimized, "ko", ms=5)
ax.set_xlabel("SASA")
ax.set_ylabel("Dipole")
fix_borders(ax)

## Is there a problem looking at the pre-optimization conformer?

In [ ]:
fig, ax = get_plot(n_ax=1)

ax.plot(*shapes_unfiltered, "go", alpha=0.5)
ax.plot(*shapes_unfiltered_optimized, "k.", alpha=0.2)

shapes_diff = shapes_unfiltered_optimized - shapes_unfiltered

ax.quiver(
    *shapes_unfiltered,
    *shapes_diff,
    alpha=0.15,
    zorder=10,
    angles='xy', scale_units='xy', scale=1
)

ax.set_xlabel("SASA")
ax.set_ylabel("Dipole")
ax.set_title("Conformer movement in shape space with optimization")

fix_borders(ax)

answer: Maybe? It's an approximation